<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_007_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational AI - aka Chatbot!

### Install Libraries

In [18]:
# !pip install python-dotenv
# !pip install openai
# !pip install google-generativeai
# !pip install anthropic
# !pip install gradio

### Import Libraries

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

### Write API Keys to .env file

In [3]:
# Path to the .env file
env_file_path = '/content/API_KEYS.env' # naming file makes it visible, no name makes it hidden

# Your OpenAI API key
OPEN_API_KEY = "sk-proj-e1GUWruINPRnro###"
ANTHROPIC_API_KEY = "sk-ant-api03-NX48###"
GOOGLE_API_KEY = "AIzaS###"

# Create the .env file and write the API keys
with open(env_file_path, 'w') as f:
    f.write(f"OPENAI_API_KEY={OPEN_API_KEY}\n")
    f.write(f"ANTHROPIC_API_KEY={ANTHROPIC_API_KEY}\n")
    f.write(f"GOOGLE_API_KEY={GOOGLE_API_KEY}\n")

print(f".env file created at: {env_file_path}")
# List all files (including hidden ones) in the /content/ folder
!ls -la /content/

.env file created at: /content/API_KEYS.env
total 20
drwxr-xr-x 1 root root 4096 Oct 22 21:35 .
drwxr-xr-x 1 root root 4096 Oct 22 21:29 ..
-rw-r--r-- 1 root root  362 Oct 22 21:35 API_KEYS.env
drwxr-xr-x 4 root root 4096 Oct 21 13:22 .config
drwxr-xr-x 1 root root 4096 Oct 21 13:22 sample_data


### Load Environment Variables

In [4]:
# Load the environment variables from the .env file
load_dotenv('/content/API_KEYS.env')  # Ensure this is the correct path to your file

# Get the API keys from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if the keys are loaded correctly and print a portion of them
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[0:10]}...")  # Only print part of the key
else:
    print("OpenAI API key not loaded correctly.")

if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[0:10]}...")
else:
    print("Anthropic API key not loaded correctly.")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[0:10]}...")
else:
    print("Google API key not loaded correctly.")

OpenAI API Key loaded: sk-proj-e1...
Anthropic API Key loaded: sk-ant-api...
Google API Key loaded: AIzaSyDh3a...


In [5]:
import openai
import anthropic
import google.generativeai

# Connect to OpenAI
openai.api_key = openai_api_key  # Set OpenAI API key

# Connect to Anthropic (Claude)
claude = anthropic.Anthropic(api_key=anthropic_api_key)  # Set Anthropic API key

# Connect to Google Generative AI
google.generativeai.configure(api_key=google_api_key)  # Set Google API key

## Message Structure for OpenAI

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```

### Major Concepts You Should Be Learning:

1. **Conversation History in ChatGPT**:
   - **How history is handled**: The `history` parameter contains the conversation history between the user and the assistant. This history is crucial because ChatGPT doesn't maintain state across calls. So, the full conversation context needs to be passed each time.
   - **Rebuilding the conversation**: The `for` loop iterates over the history, adding past user messages (`"role": "user"`) and assistant responses (`"role": "assistant"`) to the `messages` list. This maintains the continuity of the conversation by passing the entire message history to the model in each API call.

2. **System and User Messages**:
   - **System message**: As before, the system message sets the tone for the assistant's behavior. It’s added first in the `messages` list.
   - **User messages**: The current user message (along with the conversation history) is added to the `messages` list. This helps provide context to the model about the ongoing interaction.

3. **Streaming Responses**:
   - **Streamed output**: The `stream=True` parameter enables response streaming. Instead of waiting for the entire response, you get real-time chunks of the model's output.
   - **Yielding the response**: The response is constructed incrementally by appending each chunk of text to `response` and then yielding the partial response. This allows you to update the UI (or any interface) in real time as the model generates the response.

4. **Handling Role-based Messages**:
   - **Assistant and user roles**: The API expects messages to have a role. This function adds `"role": "user"` for user inputs and `"role": "assistant"` for the model's previous outputs. By doing this, the API understands the flow of conversation and can generate an appropriate response based on both the assistant's past replies and the user’s queries.

### Key Differences from Previous Code:
- **Conversation History**: This function introduces conversation history, which is a key feature in creating more dynamic and context-aware interactions with the model. The history is structured in pairs (user message, assistant message) and is passed along to give context to the assistant for every new input.
- **Role Assignment**: In previous examples, we only passed the system and user messages. Here, the function handles both user and assistant roles, ensuring the model can respond in context to previous interactions.

### What You Should Focus On:

1. **Maintaining Conversation State**:
   - You need to understand how the conversation history is passed in and rebuilt for every new request. This is important because ChatGPT doesn't remember conversations from previous API calls unless you include the history explicitly.
   
2. **Streaming with Real-time Updates**:
   - Streaming is used here to yield partial responses from the model as they are generated. This is useful for creating a more responsive and dynamic user experience, especially for longer outputs.

3. **Role-based Messages**:
   - The model needs to know who is saying what. The `"role"` field ensures the model can differentiate between the user’s inputs and the assistant’s responses, which is critical for maintaining the conversational flow.



## Chat Function

In [13]:
MODEL = 'gpt-4o-mini'

system_message = "You are a helpful assistant"

def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# And then enter Gradio's magic!
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a58f7531221647e7a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### History Dictionary vs List

The deprecation warning you're seeing indicates that Gradio has updated the way it handles messages in its chatbot interface. Specifically, Gradio no longer supports passing conversation history as tuples (like `("user message", "assistant message")`). Instead, it now expects messages in a format similar to OpenAI's API: a list of dictionaries with `"role"` and `"content"` keys.

### Key Action:

Make sure the history is structured as a list of dictionaries, each with `"role"` and `"content"`. Gradio expects this format going forward.


### Changes:
1. **History Format**: Ensure `history` is a list of dictionaries, where each message has the `"role"` (either `"user"` or `"assistant"`) and `"content"` (the actual message). This aligns with both OpenAI's API and Gradio's new expected message format.
2. **No Tuples**: Avoid using tuples like `("user message", "assistant message")`. Instead, each message should be structured with the keys `"role"` and `"content"`, as in `{"role": "user", "content": user_message}`.

### Example of History Format:
Here’s an example of how the conversation history should now look:
```python
history = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great! How can I assist you today?"},
]
```

The main difference in the code is how the **conversation history** is structured and processed. In the previous version of the code, the **history** was expected to be a list of tuples (e.g., `("user message", "assistant message")`), whereas now Gradio expects the history to be passed as a list of dictionaries with `role` and `content` keys, matching OpenAI’s message structure.

### Previous Code (Using Tuples for History):

```python
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        # Adding tuples directly, user-assistant pairs
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})
    # Streaming response logic...
```

### Updated Code (Using `role` and `content` Keys in History):

```python
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for msg in history:
        # Now history is a list of dictionaries with role and content
        messages.append(msg)
    messages.append({"role": "user", "content": message})
    # Streaming response logic...
```

### Key Code Difference:
- **Previous Code**: The history was processed as **tuples**: `(user_message, assistant_message)`, requiring two `append()` calls (one for the user and one for the assistant).
- **Updated Code**: The history is now processed as a **list of dictionaries** with `role` and `content` fields, so each dictionary is directly appended to the `messages` list in one go.



## Chat Function with Shopping Assistant

In [14]:
import gradio as gr

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful assistant"

def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding user and assistant messages using OpenAI-style format
    for msg in history:
        # Expecting history in OpenAI-style with role and content
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    # Send the message to the OpenAI API with streaming
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# More detailed system message for the assistant behavior
system_message = (
    "You are a helpful assistant in a clothing store. Your goal is to gently encourage "
    "the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. "
    "For example, if the customer says, 'I'm looking to buy a hat,' you could reply with something like, "
    "'Wonderful - we have lots of hats, including several that are part of our sales event.' "
    "Encourage the customer to buy hats if they are unsure what to get."
)

# Specify type='messages' to use OpenAI-style role/content format
gr.ChatInterface(fn=chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5388d39abcbe39c7fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

### Alternate System Role

In [16]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://631ec93f56457e6092.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Add Context

In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    if 'belt' in message: # add context about belts
        messages.append({"role": "system", "content": "For added context, the store does not sell belts, \
                        but be sure to point out other items on sale"})

    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e82b3990fef5554d51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
